In [1]:
import os

In [2]:
# check current working directory
os.getcwd() 

'C:\\Users\\fanfan\\Documents\\Capstone\\DSCI_591_capstone-BCStats\\notebooks'

In [3]:
# change the working directory to the root directory of the project
os.chdir('../')
os.getcwd()

'C:\\Users\\fanfan\\Documents\\Capstone\\DSCI_591_capstone-BCStats'

In [73]:
import pandas as pd
import numpy as np
#import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from skmultilearn.problem_transform import BinaryRelevance

In [7]:
# read in 2018 qualitative data
df_2018 = pd.read_csv("data/interim/train_2018-qualitative-data.csv")

df_2018 = df_2018[['2018 Comment']].join(df_2018.loc[:,'CPD':'OTH'])
df_2018 = df_2018.rename(columns = {'2018 Comment' : 'comment'})

categories = df_2018.loc[:,'CPD':'OTH'].columns.tolist()

In [42]:
categories

['CPD',
 'CB',
 'EWC',
 'Exec',
 'FWE',
 'SP',
 'RE',
 'Sup',
 'SW',
 'TEPE',
 'VMG',
 'OTH']

In [9]:
df_2018.head()

,comment,CPD,CB,EWC,Exec,FWE,SP,RE,Sup,SW,TEPE,VMG,OTH
0,I would suggest having a developmental growth ...,1,0,0,0,0,1,0,0,0,0,0,0
1,Base decisions regarding fish and wildlife on ...,0,0,0,0,0,0,0,0,0,0,1,0
2,Get rid of Leading Workplace Strategies and gi...,0,0,0,0,1,0,0,0,0,0,0,0
3,We are the lowest paid in Canada with a worklo...,0,1,0,0,0,1,0,0,0,0,0,0
4,Official acknowledgement of the limited divers...,0,0,1,0,0,1,0,0,0,0,0,0


In [8]:
from src.data.preprocessing_text import (
    clean_text, clean_numbers, replace_typical_misspell, remove_stopwords,
    balance_themes
)

In [56]:
from src.features.word_vectors import (
    build_vocab, check_coverage, get_average_embeddings
)

In [76]:
from src.models.eval import theme_results

#### Preprocessing

In [29]:
# get the vacabulary size of the glove_twitter_model
print("the vocabulary size in FastText model is",len(glove_twitter_model.wv.vocab))

# Preprocessing Data -- Remove punctuation, clean numbers, and fix spelling
df_2018["comment"] = df_2018["comment"].apply(lambda x: clean_text(x))

df_2018["comment"] = df_2018["comment"].apply(lambda x: clean_numbers(x))

df_2018["comment"] = df_2018["comment"].apply(
    lambda x: replace_typical_misspell(x)
)

sentences = df_2018["comment"].str.split()
sentences = remove_stopwords(sentences)

###  GloVe embeddings -- Twitter

Convert the GloVe embeddings to the Word2Vec format using the following command. 
>python -m gensim.scripts.glove2word2vec -i "glove.twitter.27B.200d.txt" -o "glove.twitter.27B.200d.w2v.txt"

In [88]:
from gensim.models import KeyedVectors
glove_twitter_model = KeyedVectors.load_word2vec_format(
    'references/pretrained_embeddings/glove.twitter.27B/glove.twitter.27B.200d.w2v.txt',
    binary=False)  # C text format

In [89]:
# get the vacabulary size of the glove_twitter_model
print("the vocabulary size in FastText model is",len(glove_twitter_model.wv.vocab))

C:\Users\fanfan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


the vocabulary size in FastText model is 1193514


In [65]:
# Build average word vectors to train on
X_wv_twitter = np.array([get_average_embeddings(sentence, embeddings_index=glove_twitter_model,
                                                n_features=200) for sentence in sentences])
Y_twitter = np.array(df_2018.loc[:,"CPD":"OTH"])

X_train_twitter, X_valid_twitter, Y_train_twitter, Y_valid_twitter = train_test_split(X_wv_twitter, Y_twitter, 
                                                            test_size=0.25, 
                                                            random_state=2019)

In [67]:
X_train_twitter.shape

(9958, 200)

In [68]:
X_valid_twitter.shape

(3320, 200)

In [69]:
Y_train_twitter.shape

(9958, 12)

In [70]:
Y_valid_twitter.shape

(3320, 12)

#### LinearSVC

In [75]:
classifier = BinaryRelevance(classifier=LinearSVC())
classifier.fit(X_train_twitter,Y_train_twitter)
prediction = classifier.predict(X_valid_twitter).toarray()

C:\Users\fanfan\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\fanfan\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\fanfan\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\fanfan\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\fanfan\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iteration

In [77]:
theme_results(Y_valid_twitter,prediction)

Overall Accuracy: 0.3178 
Hamming Loss: 0.0913 
Hamming Loss (pred. zeros): 0.1191


,Label,Y_proportion,Pred_proportion,Error,Dummy_Diff,Accuarcy,Precision,Recall
0,CPD,0.127410,0.056024,0.099096,0.028313,0.900904,0.752688,0.330969
1,CB,0.184639,0.126807,0.089157,0.095482,0.910843,0.876485,0.601958
2,EWC,0.084337,0.013554,0.079217,0.005120,0.920783,0.688889,0.110714
3,Exec,0.103012,0.017470,0.098795,0.004217,0.901205,0.620690,0.105263
4,FWE,0.062048,0.025602,0.047892,0.014157,0.952108,0.776471,0.320388
5,SP,0.096386,0.025301,0.087952,0.008434,0.912048,0.666667,0.175000
6,RE,0.085542,0.008735,0.087048,-0.001506,0.912952,0.413793,0.042254
7,Sup,0.127711,0.029217,0.119578,0.008133,0.880422,0.639175,0.146226
8,SW,0.165964,0.060843,0.149096,0.016867,0.850904,0.638614,0.234120
9,TEPE,0.228614,0.193373,0.091265,0.137349,0.908735,0.855140,0.723320


###  GloVe embeddings -- Wikipedia 2014 + Gigaword 5 

Convert the GloVe embeddings to the Word2Vec format using the following command.
>python -m gensim.scripts.glove2word2vec -i "glove.6B.300d.txt" -o "glove.6B.300d.w2v.txt"

In [81]:
glove_6B_model = KeyedVectors.load_word2vec_format(
    'references/pretrained_embeddings/glove.6B/glove.6B.300d.w2v.txt', 
    binary=False)  # C text format

In [82]:
# get the vacabulary size of the glove_6B_model
print("the vocabulary size in FastText model is",len(glove_6B_model.wv.vocab))

C:\Users\fanfan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


the vocabulary size in FastText model is 400000


In [85]:
# Build average word vectors to train on
X_wv_6B = np.array([get_average_embeddings(sentence, embeddings_index=glove_6B_model, n_features=300)
                 for sentence in sentences])
Y_6B = np.array(df_2018.loc[:,"CPD":"OTH"])

X_train_6B, X_valid_6B, Y_train_6B, Y_valid_6B = train_test_split(X_wv_6B, Y_6B, 
                                                            test_size=0.25, 
                                                            random_state=2019)

In [86]:
classifier = BinaryRelevance(classifier=LinearSVC())
classifier.fit(X_train_6B,Y_train_6B)
prediction = classifier.predict(X_valid_6B).toarray()

In [87]:
theme_results(Y_valid_6B,prediction)

Overall Accuracy: 0.3545 
Hamming Loss: 0.0874 
Hamming Loss (pred. zeros): 0.1191


,Label,Y_proportion,Pred_proportion,Error,Dummy_Diff,Accuarcy,Precision,Recall
0,CPD,0.127410,0.060843,0.095482,0.031928,0.904518,0.762376,0.364066
1,CB,0.184639,0.134036,0.083133,0.101506,0.916867,0.878652,0.637847
2,EWC,0.084337,0.016566,0.076807,0.007530,0.923193,0.727273,0.142857
3,Exec,0.103012,0.021988,0.099096,0.003916,0.900904,0.589041,0.125731
4,FWE,0.062048,0.027711,0.046386,0.015663,0.953614,0.782609,0.349515
5,SP,0.096386,0.038855,0.078614,0.017771,0.921386,0.728682,0.293750
6,RE,0.085542,0.010241,0.081325,0.004217,0.918675,0.705882,0.084507
7,Sup,0.127711,0.037048,0.116566,0.011145,0.883434,0.650407,0.188679
8,SW,0.165964,0.071687,0.140060,0.025904,0.859940,0.680672,0.294011
9,TEPE,0.228614,0.192169,0.091867,0.136747,0.908133,0.855799,0.719368


###  GloVe embeddings -- Common Crawl

Convert the GloVe embeddings to the Word2Vec format using the following command.

>python -m gensim.scripts.glove2word2vec -i "glove.840B.300d.txt" -o "glove.840B.300d.w2v.txt"

In [90]:
glove_840B_model = KeyedVectors.load_word2vec_format(
    'references/pretrained_embeddings/glove.840B.300d/glove.840B.300d.w2v.txt', 
    binary=False)  # C text format

In [92]:
# get the vacabulary size of the glove_840B_model
print("the vocabulary size in FastText model is",len(glove_840B_model.wv.vocab))

C:\Users\fanfan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


the vocabulary size in FastText model is 2196016


In [93]:
# Build average word vectors to train on
X_wv_840B = np.array([get_average_embeddings(sentence, embeddings_index=glove_840B_model, 
                                             n_features=300) for sentence in sentences])
Y_840B = np.array(df_2018.loc[:,"CPD":"OTH"])

X_train_840B, X_valid_840B, Y_train_840B, Y_valid_840B = train_test_split(X_wv_840B, Y_840B, 
                                                            test_size=0.25, 
                                                            random_state=2019)

In [94]:
classifier = BinaryRelevance(classifier=LinearSVC())
classifier.fit(X_train_840B,Y_train_840B)
prediction = classifier.predict(X_valid_840B).toarray()

In [95]:
theme_results(Y_valid_840B,prediction)

Overall Accuracy: 0.3946 
Hamming Loss: 0.0822 
Hamming Loss (pred. zeros): 0.1191


,Label,Y_proportion,Pred_proportion,Error,Dummy_Diff,Accuarcy,Precision,Recall
0,CPD,0.127410,0.068072,0.090060,0.037349,0.909940,0.774336,0.413712
1,CB,0.184639,0.146084,0.072289,0.112349,0.927711,0.884536,0.699837
2,EWC,0.084337,0.023795,0.076205,0.008133,0.923795,0.670886,0.189286
3,Exec,0.103012,0.043675,0.084639,0.018373,0.915361,0.710345,0.301170
4,FWE,0.062048,0.040361,0.041566,0.020482,0.958434,0.753731,0.490291
5,SP,0.096386,0.041566,0.079518,0.016867,0.920482,0.702899,0.303125
6,RE,0.085542,0.015060,0.080723,0.004819,0.919277,0.660000,0.116197
7,Sup,0.127711,0.048795,0.113855,0.013855,0.886145,0.641975,0.245283
8,SW,0.165964,0.080723,0.132831,0.033133,0.867169,0.705224,0.343013
9,TEPE,0.228614,0.203313,0.081928,0.146687,0.918072,0.860741,0.765481
